In [1]:
import math
import qiskit_qasm2
from qiskit_qasm2 import load, CustomClassical
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
from scipy.linalg import expm
from numpy.linalg import norm, matrix_power
from functools import reduce
from itertools import product
from qiskit import QuantumCircuit
import qiskit.quantum_info as qi
from matplotlib.ticker import MaxNLocator
import time

from os import listdir
from os.path import isfile, join

PAULIS = {'I': np.eye(2, dtype='complex'),
          'X': np.array([[0, 1], [1, 0]], dtype='complex'),
          'Y': np.array([[0, -1j], [1j, 0]]),
          'Z': np.diag(np.array([1, -1], dtype='complex'))}

dict_Walsh = {'00': 'I',
              '01': 'Z',
              '10': 'X',
              '11': 'Y'}


def speed_multiple(b,c_map):
    """Multiply each column of a matrix by given constant."""
    for j, c in enumerate(c_map):
        b[:,j] *= c
    return b


def make_B(N, coefs, cbase):
    """ make matrix B s.t. L = -BB.T with correct zero boundary conditions.
    N - size of B.
    coefs - list of coefficients.
    cbase - divider.
    
    """
    B = np.zeros((N,N))
    l = len(coefs)//2
    diags = list(range(-l,l+1))
    for i,val in zip(diags,coefs):
        B += val*np.eye(N,k=i)

    ### Boundary connditions
    ## left side
    B[0,:] = np.zeros_like(B[0,:])
    B[:,0] = np.zeros_like(B[:,0])
    
    for i in range(l):
        coef_temp = coefs[-(i+1):]
        B[l-i,:len(coef_temp)] = B[l-i,:len(coef_temp)] - coef_temp
        # print(l-i, coef_temp)
    B[:,0] = B[:,0]*np.sqrt(2)

    ## right side
    B[-1,:] = np.zeros_like(B[-1,:])
    B[:,-1] = np.zeros_like(B[:,-1])
    for i in range(l):
        coef_temp = coefs[:(i+1)]
        B[-(l-i)-1,-len(coef_temp):] = B[-(l-i)-1,-len(coef_temp):] - coef_temp
        # print(-(l-i)-1, coef_temp)
    B[:,-1] = B[:,-1]*np.sqrt(2)
    
    # return B
    return B/cbase


def get_pauli_from_xz(x,z,n):
    """ Return Pauli string and a coefficient given
    x - integer number 
    z - integer number 
    n - lenght of binary representation
    """
    a = bin(x & z).count("1")
    p_str = ''.join([dict_Walsh[i+j] for i,j in zip(f'{x:0{n}b}',f'{z:0{n}b}')])
    b = p_str.count('Z')
    return 1j**(a+3*b), p_str

def get_wwH(x, z, H):
    N = H.shape[0]
    a = bin(x & z).count("1")
    beta = sum([(-1)**(bin(z & p).count("1"))*H[p,x ^ p] for p in range(N)]) # bottleneck! list contain a lot of zeros!
    return 1j**(a)*beta/N

In [2]:
NQ = 9
nq = NQ-1


accuracy = 6
cbase = 60                    # normalization of the coefficients from fixed format representation
coefs = [-1, 9, -45, 0, 45, -9, 1]        # list of the discretization coefficients, ncoef of them

B = make_B(2**(nq),coefs,cbase)
H = np.block([[np.zeros_like(B), B],[B.T, np.zeros_like(B)]])

In [3]:
t_trotter_comm_group = 1

def W(x, z, dummy):
    """
    Returns angle for rz
    x - x-stirng or group-id
    z - z-stirng
    dummy - cbase (normalization for finite difference scheme)
    example of rz: rz(w(10,10,60))
    """
    # global c_weight
    x = int(x)
    z = int(z)
    wwH = get_wwH(x, z, H)
    weights_from_func_W.append(wwH)
    rz_angle = 2*t_trotter_comm_group*wwH
    return rz_angle.real

customs = [
    CustomClassical("w", 3, W),
]

In [4]:
mypath = 'qdecomp_v5/QASM_files'
file_names = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f)) and '-{}_{}_'.format(accuracy,NQ) in f]
dict_sets = dict(zip(range(len(file_names)),file_names))


for filename in file_names:
    print(filename)
    """Read Pauli strings and coefficients (signs) from file """
    p_str_list = []
    p_str_list_diag = []
    w_coef_dict = {}

    with open(filename) as f:
        counter = 0 # to count lines starting with //
        for line in f:
            line = line.strip()
            if 'Differential coefficients' in line:
                counter = 1

            if counter == 1: # diagonal part (Rz)
                if 'rz(' in line:
                    temp = line.split()[2] # //XIII,+ZIII
                    p_str_d = temp[-NQ-1:] # e.g. +IZIZI
                    p_str = temp[2:NQ+2] # e.g. XYXYX

                    temp = line.split()[0]
                    w_coef = tuple([int(i) for i in temp[5:-2].split(',')])# e.g. (10,2,60)

                    p_str_list.append(p_str)
                    p_str_list_diag.append(p_str_d)
                    w_coef_dict[w_coef] = p_str
    
    """Read circuit from QASM file """
    
    weights_from_func_W = []
    start = time.time()
    circuit_l = load(filename, custom_classical=customs).reverse_bits()
    end = time.time()
    print("\tTime to read the circuit: {:.3f} s".format(end-start))
    matrix_l = qi.Operator(circuit_l).data

    """Compare results"""
    sum_error_coef = 0
    
    for p_str, p_str_d, wwH in zip(p_str_list,p_str_list_diag, weights_from_func_W):
        P = reduce(np.kron, [PAULIS[s] for s in p_str])
        alpha_trace = np.trace(H@P)/(2**NQ) # compute coefficient as inner product
#         print('\ttrace = {}, get_wwH = {}'.format(alpha_trace, wwH))
        error = norm(alpha_trace - wwH)
        assert error < 10**(-10)
        sum_error_coef+=error
    print('\tNorm of difference for every coef is < 10^(-10)')
    print('\tSum of norms for the group is {:.3E}'.format(sum_error_coef))

qdecomp_v5/QASM_files/hdecomp-6_9_510_GD.qasm
	Time to read the circuit: 0.046 s
	Norm of difference for every coef is < 10^(-10)
	Sum of norms for the group is 0.000E+00
qdecomp_v5/QASM_files/hdecomp-6_9_270_GD.qasm
	Time to read the circuit: 0.044 s
	Norm of difference for every coef is < 10^(-10)
	Sum of norms for the group is 4.120E-17
qdecomp_v5/QASM_files/hdecomp-6_9_318_GD.qasm
	Time to read the circuit: 0.044 s
	Norm of difference for every coef is < 10^(-10)
	Sum of norms for the group is 3.296E-17
qdecomp_v5/QASM_files/hdecomp-6_9_256_GD.qasm
	Time to read the circuit: 0.085 s
	Norm of difference for every coef is < 10^(-10)
	Sum of norms for the group is 0.000E+00
qdecomp_v5/QASM_files/hdecomp-6_9_509_GD.qasm
	Time to read the circuit: 0.044 s
	Norm of difference for every coef is < 10^(-10)
	Sum of norms for the group is 0.000E+00
qdecomp_v5/QASM_files/hdecomp-6_9_269_GD.qasm
	Time to read the circuit: 0.044 s
	Norm of difference for every coef is < 10^(-10)
	Sum of norms f